In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import StackingClassifier

In [2]:
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_data.pqt")
test_df_2 = pd.read_parquet("test_data.pqt")

In [3]:
train_df.head(3)

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other}


In [4]:
test_df.head(3)

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster
0,200000,month_4,-0.096224,0.335496,-0.125995,-0.095578,channel_code_12,city_14,city_type_0,None,...,0.010952,0.946066,0.407762,-0.153950,0.548895,0.541020,0.031742,0.257278,0.561353,{α}
1,200000,month_5,-0.024255,-0.059806,-0.124295,-0.023381,channel_code_12,city_14,city_type_0,None,...,0.006812,0.945281,0.396267,-0.150505,0.549468,0.552131,0.237817,0.264211,0.715199,{α}
2,200000,month_6,0.045988,0.049418,-0.125995,0.047079,channel_code_12,city_14,city_type_0,None,...,0.006812,0.945281,0.396267,-0.152800,0.549468,0.541020,0.387566,0.268543,0.836079,None


In [5]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year",
]
num_cols = ['id', 'balance_amt_avg', 'balance_amt_max', 'balance_amt_min',
       'balance_amt_day_avg', 'ogrn_days_end_month', 'ogrn_days_end_quarter',
       'ft_registration_date', 'max_founderpres',
       'min_founderpres', 'ogrn_exist_months', 'sum_of_paym_2m',
       'sum_of_paym_6m', 'sum_of_paym_1y', 'sum_a_oper_1m',
       'cnt_a_oper_1m', 'sum_b_oper_1m', 'cnt_b_oper_1m', 'sum_c_oper_1m',
       'cnt_c_oper_1m', 'sum_deb_d_oper_1m', 'cnt_deb_d_oper_1m',
       'sum_cred_d_oper_1m', 'cnt_cred_d_oper_1m', 'sum_deb_e_oper_1m',
       'cnt_deb_e_oper_1m', 'cnt_days_deb_e_oper_1m', 'sum_cred_e_oper_1m',
       'cnt_cred_e_oper_1m', 'cnt_days_cred_e_oper_1m', 'sum_deb_f_oper_1m',
       'cnt_deb_f_oper_1m', 'cnt_days_deb_f_oper_1m', 'sum_cred_f_oper_1m',
       'cnt_cred_f_oper_1m', 'cnt_days_cred_f_oper_1m', 'sum_deb_g_oper_1m',
       'cnt_deb_g_oper_1m', 'cnt_days_deb_g_oper_1m', 'sum_cred_g_oper_1m',
       'cnt_cred_g_oper_1m', 'cnt_days_cred_g_oper_1m', 'sum_deb_h_oper_1m',
       'cnt_deb_h_oper_1m', 'cnt_days_deb_h_oper_1m', 'sum_cred_h_oper_1m',
       'cnt_cred_h_oper_1m', 'cnt_days_cred_h_oper_1m', 'sum_a_oper_3m',
       'cnt_a_oper_3m', 'sum_b_oper_3m', 'cnt_b_oper_3m', 'sum_c_oper_3m',
       'cnt_c_oper_3m', 'sum_deb_d_oper_3m', 'cnt_deb_d_oper_3m',
       'sum_cred_d_oper_3m', 'cnt_cred_d_oper_3m', 'sum_deb_e_oper_3m',
       'cnt_deb_e_oper_3m', 'cnt_days_deb_e_oper_3m', 'sum_cred_e_oper_3m',
       'cnt_cred_e_oper_3m', 'cnt_days_cred_e_oper_3m', 'sum_deb_f_oper_3m',
       'cnt_deb_f_oper_3m', 'cnt_days_deb_f_oper_3m', 'sum_cred_f_oper_3m',
       'cnt_cred_f_oper_3m', 'cnt_days_cred_f_oper_3m', 'sum_deb_g_oper_3m',
       'cnt_deb_g_oper_3m', 'cnt_days_deb_g_oper_3m', 'sum_cred_g_oper_3m',
       'cnt_cred_g_oper_3m', 'cnt_days_cred_g_oper_3m', 'sum_deb_h_oper_3m',
       'cnt_deb_h_oper_3m', 'cnt_days_deb_h_oper_3m', 'sum_cred_h_oper_3m',
       'cnt_cred_h_oper_3m', 'cnt_days_cred_h_oper_3m']

In [6]:
for column in num_cols:
    train_df[column].fillna(train_df[column].mean(), inplace=True)
for column in cat_cols:
    train_df[column].fillna(train_df[column].mode()[0], inplace=True)

In [7]:
test_train_df = train_df.drop(cat_cols, axis=1)
test_train_df = test_train_df.drop(["date", "end_cluster"], axis=1)

In [8]:
matrix = test_train_df.corr().abs() # создаем матрицу корреляций

threshold = 0.7 # порог можно поставить любой,
                # для примера возьмем 0.7, однако, можно поэкспериментировать
                # с разными значениями

upper = matrix.where(np.triu(np.ones(matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
catboost_df = test_train_df.drop(to_drop, axis=1)

In [9]:
# I prepared a function for reviewing the dataset.
def Df_info(df):
    print("########### Head ###########")
    print(df.head())
    print("########### Tail ###########")
    print(df.tail())
    print("########### Shape ###########")
    print(df.shape)
    print("########### Info ###########")
    print(df.info())
    print("########### Info ###########")
    print(df.columns)
    print("########### Quantiles ###########")
    print(df.describe().T)
    print("########### NA ###########")
    print(df.isnull().sum())

In [10]:
catboost_df[cat_cols] = train_df[cat_cols]

# Df_info(catboost_df)

In [11]:
Df_info(catboost_df)

########### Head ###########
   id  balance_amt_avg  balance_amt_min  ogrn_days_end_month  \
0   0         0.744845         1.287207            -0.488553   
1   0         1.049605         2.458609            -0.488553   
2   0         0.692653         0.430042            -0.488553   
3   1        -0.081586        -0.114040             0.324343   
4   1        -0.094962        -0.119302             0.324343   

   ogrn_days_end_quarter  ft_registration_date  max_founderpres  \
0              -0.135063              2.452182     2.736461e+00   
1              -0.135063              2.469917     2.756014e+00   
2              -0.135063              2.485936     2.773674e+00   
3               1.258747              0.174035     1.211971e-17   
4               1.258747              0.191770     1.211971e-17   

   sum_of_paym_2m  sum_a_oper_1m  cnt_a_oper_1m  ...  cnt_b_oper_3m  \
0        0.942275      -0.188835       0.417022  ...       0.517298   
1        0.645704      -0.188835       0.

In [12]:
for column in num_cols:
    test_df[column].fillna(test_df[column].mean(), inplace=True)
for column in cat_cols[:-1]:
    test_df[column].fillna(test_df[column].mode()[0], inplace=True)

In [13]:
catboost_df["end_cluster"] = train_df["end_cluster"]

In [14]:
catboost_df.shape

(600000, 44)

In [15]:
test_df = test_df[catboost_df.columns[:-1]]
test_df.shape

(290120, 43)

In [16]:
catboost_df[cat_cols] = catboost_df[cat_cols].astype("str")
test_df[cat_cols] = test_df[cat_cols].astype("str")

In [17]:
X = catboost_df.drop(["id", "end_cluster"], axis=1)
y = catboost_df["end_cluster"]

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [18]:
Df_info(catboost_df)

########### Head ###########
   id  balance_amt_avg  balance_amt_min  ogrn_days_end_month  \
0   0         0.744845         1.287207            -0.488553   
1   0         1.049605         2.458609            -0.488553   
2   0         0.692653         0.430042            -0.488553   
3   1        -0.081586        -0.114040             0.324343   
4   1        -0.094962        -0.119302             0.324343   

   ogrn_days_end_quarter  ft_registration_date  max_founderpres  \
0              -0.135063              2.452182     2.736461e+00   
1              -0.135063              2.469917     2.756014e+00   
2              -0.135063              2.485936     2.773674e+00   
3               1.258747              0.174035     1.211971e-17   
4               1.258747              0.191770     1.211971e-17   

   sum_of_paym_2m  sum_a_oper_1m  cnt_a_oper_1m  ...    channel_code     city  \
0        0.942275      -0.188835       0.417022  ...  channel_code_5  city_23   
1        0.645704    

In [19]:
model = CatBoostClassifier(iterations=1000,
                           depth=6,
                           task_type="GPU",
                           devices='0:1')
model.fit(x_train, y_train, cat_features=cat_cols)

Learning rate set to 0.228695
0:	learn: 1.4006469	total: 60.7ms	remaining: 1m
1:	learn: 1.2601370	total: 123ms	remaining: 1m 1s
2:	learn: 1.1695431	total: 188ms	remaining: 1m 2s
3:	learn: 1.1038781	total: 251ms	remaining: 1m 2s
4:	learn: 1.0552165	total: 314ms	remaining: 1m 2s
5:	learn: 1.0173161	total: 379ms	remaining: 1m 2s
6:	learn: 0.9894268	total: 446ms	remaining: 1m 3s
7:	learn: 0.9660727	total: 512ms	remaining: 1m 3s
8:	learn: 0.9499488	total: 580ms	remaining: 1m 3s
9:	learn: 0.9347693	total: 637ms	remaining: 1m 3s
10:	learn: 0.9247220	total: 703ms	remaining: 1m 3s
11:	learn: 0.9160324	total: 760ms	remaining: 1m 2s
12:	learn: 0.9087982	total: 825ms	remaining: 1m 2s
13:	learn: 0.9029315	total: 883ms	remaining: 1m 2s
14:	learn: 0.8980441	total: 939ms	remaining: 1m 1s
15:	learn: 0.8942765	total: 1s	remaining: 1m 1s
16:	learn: 0.8912425	total: 1.06s	remaining: 1m 1s
17:	learn: 0.8873431	total: 1.12s	remaining: 1m 1s
18:	learn: 0.8836893	total: 1.19s	remaining: 1m 1s
19:	learn: 0.881

In [20]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [21]:
cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [22]:
y_pred_proba = model.predict_proba(x_val)
y_pred_proba.shape

(120000, 17)

In [23]:
weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)

0.9378192402271075

In [24]:
test_df_2.pivot(index="id", columns="date", values="start_cluster").head(3)

date,month_4,month_5,month_6
id,,,
200000,{α},{α},None
200001,{α},{α},None
200002,{other},{other},None


In [25]:
test_df_2["start_cluster"] = catboost_df["start_cluster"].mode()[0]
test_df_2["start_cluster"] = test_df_2["start_cluster"].astype("category")

In [26]:
sample_submission_df = pd.read_csv("sample_submission.csv")

In [27]:
sample_submission_df.shape

(100000, 18)

In [28]:
sample_submission_df.head()

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
1,200001,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
2,200002,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
3,200003,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
4,200004,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


In [29]:
test_df['date'] =test_df_2["date"]

last_m_test_df = test_df[test_df["date"] == "month_6"]
last_m_test_df = last_m_test_df.drop(["id", "date"], axis=1)

In [30]:
test_pred_proba = model.predict_proba(last_m_test_df)
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

In [31]:
test_pred_proba_df.shape

(100000, 17)

In [32]:
test_pred_proba_df.head(2)

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,0.005147,0.027717,0.013145,0.012869,0.001848,0.000013,0.000135,5.911749e-05,0.003954,0.001776,0.005569,0.000184,0.000578,4.105285e-07,0.001079,0.925926,3.800488e-07
1,0.003690,0.608799,0.000392,0.000631,0.000457,0.000020,0.000010,2.794296e-07,0.000437,0.002219,0.000267,0.000027,0.000131,1.003349e-08,0.000320,0.382546,5.216470e-05


In [33]:
sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("output.csv", index=False)